In [ ]:
import numpy as np

# Para tratamiento y e/s de datos
import pandas as pd

# Gráficos de datos
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go

#filtrado para suavizar los datos
from scipy.signal import savgol_filter

In [ ]:
from fbprophet.plot import add_changepoints_to_plot
from fbprophet.diagnostics import cross_validation
from fbprophet.diagnostics import performance_metrics
from fbprophet.plot import plot_cross_validation_metric

# Forecasting Demanda Energía (Prophet)

In [ ]:
# Importo el archivos de datos de consumo de energia activa.
demand = pd.read_csv(r'DIRECCIÓN  + NOMBRE.csv')
demand.drop(columns = 'terminal', inplace = True)
demand.rename(columns = {'fechahora' : 'datetime', 'demanda_activa' : 'y[kW]' }, inplace = True)

In [ ]:
#Convierto a tipo DateTimeIndex la columna "Datetime"
demand['datetime'] = pd.to_datetime(demand['datetime'])
demand.sort_values(by=['datetime'], axis = 0, ascending = True, inplace = True)
demand.reset_index(inplace = True, drop = True)

In [ ]:
if demand.index[0] == 0:
    demand = demand.drop(labels=0, axis=0)

In [ ]:
# De datos duplicados, solo se mantiene la medición más reciente. 
demand.drop_duplicates(subset = 'datetime', keep = 'last', inplace = True)

In [ ]:
# Datos sin filtrar
fig = go.Figure()
fig.add_trace(go.Scatter(x=demand['datetime'], y=demand['y[kW]'],
                         mode='lines',
                         name='Energía'))

# adjust layout
fig.update_traces(line=dict(width=0.5))
fig.show()

In [ ]:
print(demand.info())
print('\n')
print(demand['datetime'].min())
print(demand['datetime'].max())

## Limpieza de datos

### Tratamiento de espacios vacios para un grupo de datos continuos

In [ ]:
demand.set_index('datetime', inplace = True)
print(f'df.index.freq is set to: {demand.index.freq}')

<i>
Tener un dataset con frecuencia en "None" indica 
que existen datos que perdidos (missing). <br>
Para verificar lo dicho, podemos comparar con un rango de datos
custom e ininterrumpido
</i>

In [ ]:
# Custom range
data_range = pd.date_range(start = min(demand.index),
                          end = max(demand.index),
                          freq = '15min') 
#freq = '15min' indica frecuencia por hora.
#Explicación: genero un dataframe con una frecuencia horaria desde el valor minimo del index (datetime)
#del dataframe original, y con el valor máximo del index. Con esto lo que obtengo es TODO EL CALENDARIO
#sin datos perdidos. 
#Al hacer mas adelante la diferencia entre ambos dataframe, voy a obtener los "días perdidos" del dataframe original. 
# https://pandas.pydata.org/docs/user_guide/timeseries.html#timeseries-offset-aliases
data_range

In [ ]:
print(f'La diferencia de longitud entre el rango customizado de datos y nuestro dataset es {(len(data_range)-len(demand))}')

In [ ]:
#la diferencia entre ambos df indica la cantidad de valores perdidos en el df_original
print(data_range.difference(demand.index))

In [ ]:
# El siguiente comando adjunta los datos "datetime" perdidos (missing) al dataset original
# pero va a generar valores NaN para la variable Target (y[kW])
demand = demand.reindex(data_range)
print(f'La frecuencia de los datos es: {demand.index.freq}')

In [ ]:
print(f'Datos nulos: {demand.isnull().sum()}')

In [ ]:
# Llenamos estos valores blancos con valores que se encuentran en una curva lineal entre puntos de datos existentes
demand['y[kW]'].interpolate(method='linear', inplace=True)
print(f'Datos nulos: {demand.isnull().sum()}')

## Filtro savgol_filter

In [ ]:
y_filtered = demand[["y[kW]"]].apply(savgol_filter,  window_length=5, polyorder=3)

In [ ]:
# create figure
fig = go.Figure()
fig.add_trace(go.Scatter(x=demand.index,y=demand['y[kW]'],
                         mode='lines',
                         name='No Filtrada'))
fig.add_trace(go.Scatter(x=y_filtered.index, y=y_filtered['y[kW]'],
                         mode='lines', 
                         name='Filtrada'))

# adjust layout
fig.update_traces(line=dict(width=0.5))
fig.show()

## EDA

In [ ]:
df_eda = y_filtered.copy()

### Extraemos características de la variable Tiempo

In [ ]:
df_eda['dow'] = df_eda.index.day_of_week
df_eda['doy'] = df_eda.index.day_of_year
df_eda['year'] = df_eda.index.year
df_eda['month'] = df_eda.index.month
df_eda['quarter'] = df_eda.index.quarter
df_eda['hour'] = df_eda.index.hour
df_eda['weekday'] = df_eda.index.day_name()
df_eda['woy'] = df_eda.index.isocalendar().week #week of year
df_eda['dom'] = df_eda.index.day # Day of Month
df_eda['date'] = df_eda.index.date 
df_eda['season'] = df_eda['month'].apply(lambda month_number: (month_number%12 + 3)//3) # número de estación del año
# el operador aritmético // solo devuelve a parte entera de la división.
df_eda['date_and_time'] = df_eda.index 

### Graficando el consumo de energía a lo largo del tiempo

In [ ]:
#Plotting
fig = px.line(df_eda, x=['date_and_time'], y='y[kW]', title=f'Demanda kW por tiempo [{min(df_eda.year)} - {max(df_eda.year)}]')
fig.update_traces(line=dict(width=0.3))
fig.update_layout(xaxis_title='Date & Time', yaxis_title='Demanda Energía [kW]')
fig.show()

In [ ]:
# Estudiando la gráfica se observa un comportamiento con patron en temporadas (estación del año). 

In [ ]:
#Se puede y debe comparar con la gráfica expuesta en Mr.Dims para verificar. 

### Patrones de demanda

In [ ]:
#Podemos usar nuestras funciones de fecha y hora extraídas previamente 
#para ver si surgen patrones recurrentes de los datos agregados. 
#Tomemos, por ejemplo, la demanda de energía a lo largo del día para cada día de la semana:

In [ ]:
##### Gráfica de consumo anual

#grupo de datos por años
groups = df_eda['y[kW]'].groupby(pd.Grouper(freq='A'))

#Configuración de los ejes
fig, axs = plt.subplots(len(groups), 1, figsize=(15,15))


for ax, (name, group) in zip(axs, groups):
    
    #grafica
    ax.plot(pd.Series(group.values))

    ax.set_xlabel('Hora del año')
    ax.set_ylabel('Consumo total')
    ax.set_title(name.year)
    plt.subplots_adjust(hspace=0.5)

In [ ]:
### plot the monthly demand variability. Looking for seasonal effects

fig, axs = plt.subplots(1, 2, figsize=(30,10))

for ax, col in zip(axs, df_eda.columns):
    
    groups = df_eda[col].groupby(pd.Grouper(freq='M'))
    
    df = pd.DataFrame()
    
    for name, group in groups:
        df[name.month] = pd.Series(group.values)

    df.boxplot(ax=ax)
    ax.set_xlabel('Month Year')
    ax.set_ylabel('Energy Demanded MWh')
    ax.set_title(col)
    plt.subplots_adjust(hspace=1)
    
    
plt.show()

In [ ]:
# Dataframe definido para reflejar el consumo por hora en la semana, usando la mediana de energia. 
patron_1 = df_eda.groupby(['hour', 'weekday'], as_index=False).agg({'y[kW]':'median'})
# patron_1

In [ ]:
fig = px.line(patron_1, 
              x = 'hour',
              y = 'y[kW]', 
              color='weekday', 
              title='Mediana de consumo de energia por hs por día de semana ')

fig.update_layout(xaxis_title='Hour', yaxis_title='Energy Demand[kW]')

fig.show()

In [ ]:
# Dataframe definido para graficar el consumo horario por temporada del año. Mediana de la energía. 
patron_2 = df_eda.groupby(['hour', 'season'], as_index=False).agg({'y[kW]':'median'})
# patron_2

In [ ]:
fig_2 = px.line(patron_2, 
                x = 'hour',
                y = 'y[kW]', 
                color='season', 
                title='Mediana de consumo de energia por hs por estación')

fig_2.update_layout(xaxis_title='Hour', yaxis_title='Energy Demand[kW]')

fig_2.show()

In [ ]:
# Durante el verano le dan duro al aire acondicionado. 

## Partición de la serie de tiempo

In [ ]:
# Los puntos que representan datos a lo largo de una serie de tiempo pueden ser interesantes 
# en cuanto sus patrones se complementes con tendencias de subida/bajada y/o estacionalidad. 
# Según la info adquirida en el EDA esto parece ser así.

In [ ]:
print(f'El primer punto de medicion fecha/hs es: {min(y_filtered.index)}')
print(f'El último punto de medicion fecha/hs es: {max(y_filtered.index)}')

In [ ]:
df_prueba = y_filtered.copy().reset_index()
df_prueba = df_prueba.rename(columns = {'index': 'ds', 'y[kW]': 'y'})

### Opcion uno

In [ ]:
# 1 año = 35.040 muestras: (1 dia)24HS ---> 96 muestras
if (len(df_prueba)/35040 >= 3):
    train = df_prueba.loc[:len(df_prueba)-35040].reset_index(drop = True)
    test = df_prueba.loc[(len(df_prueba)-35040)+1:].reset_index(drop= True) 
    print('Train')
    print(min(train['ds']))
    print(max(train['ds']))
    print('Test')
    print(min(test['ds']))
    print(max(test['ds']))    
else:
    print('no ok')

In [ ]:
#Se permite recortar varias fechas porque:
#1- El comportamiento es constante en el tiempo.
#2- Alivia la carga de procesamiento en la PC.
print(f'Información set-train: {train.shape}\nInformación set-test: {test.shape}\n')
print(f'Proporción del test-total: {((len(test)*100)/len(df_prueba)):.2f}%\n') 

### Opcion dos

### Opcion tres

In [ ]:
futuro = make_future_dataframe(modelo_tuneado, periods = 35.040, freq = "15 min", include_history = False)

# Prophet

Es un modelo de pronóstico de series de tiempo, diseñado para manejar las características comunes
en las series de tiempo implementadas hoy en día. <br>
La idea del modelo Prophet es ser accesible y ajustable sin necesitar tener conocimientos de lo que pasa
detrás del telón respecto al funcionamiento matemático de la serie de tiempo. <br>
Tecnicamente hablando, es una serie de tiempo descompuesta en tres términos:
<i>y(t) = g(t)+s(t)+h(t)+et</i>
<ul>
<li>g(t): trend
    <blockquote> 
        Función de tendencia que modela cambios no-periodicos en los valores de la serie de tiempo.
    </blockquote>
    </li> 
<li>s(t): seasonality
    <blockquote>   
        Función que representa cambios periodicos. 
    </blockquote>
    </li> 
<li>h(t): holidays
    <blockquote>  
        Función que representa los efectos de los días de vacaciones/feriados/findes.
    </blockquote>
    </li>
<li>et: Término de error. 
    <blockquote>  
        Representa cualquier cambio idiosincracico (herencia). Se supone normalmente distribuido. 
    </blockquote>
    </li>
</ul>

Docs Oficiales (muy utiles): __[PROPHET_DOCS](https://facebook.github.io/prophet/docs/quick_start.html)__<BR>
Teoría: __[Forecasting at Scale(pdf)](https://www.kaggle.com/robinteuwens/forecasting-energy-demand/notebook)__ <br>
Practica: __[Forecasting con Prophet](https://nextjournal.com/eric-brown/forecasting-with-prophet)__ 

In [ ]:
print(train.head())
print('\n')
print(test.head())

In [ ]:
f, ax = plt.subplots(figsize=(14,5))
train.plot(kind='line', x='ds', y='y', color='blue', label='Train', ax=ax)
test.plot(kind='line', x='ds', y='y', color='red', label='Test', ax=ax)
plt.title('Energía demandada: Traning and Test data')
plt.show()

## Conditional Seasonalities

__Teoría (fundamentos): ['How does Prophet work?'](https://medium.com/analytics-vidhya/how-does-prophet-work-part-2-c47a6ceac511)__

<blockquote>
In some instances the seasonality may depend on other factors, such as a weekly seasonal pattern that is different during the summer than it is during the rest of the year, or a daily seasonal pattern that is different on weekends vs. on weekdays. These types of seasonalities can be modeled using conditional seasonalities.
</blockquote>

In [ ]:
# Del EDA podemos observar que la variación diaria en estaciones es mayor en Verano e Invierno (obviamente). 
# Destripemos los patrones de los datos para tener en cuenta la interdependencia de estas variables.

In [ ]:
# Condiciones
def is_spring(ds): 
    date = pd.to_datetime(ds)    
    return (date.month >= 3) & (date.month <=5)

def is_summer(ds): 
    date = pd.to_datetime(ds)
    return (date.month >= 6) & (date.month <=8)

def is_autumn(ds): 
    date = pd.to_datetime(ds)
    return (date.month >= 9) & (date.month <=11)

# La lógica fallaba, tuve que corregir. 
def is_winter(ds): 
    date = pd.to_datetime(ds)
    return (date.month == 12) | (date.month <=2)

# A esta función la hice de una forma distinta para que ande bien.
def is_weekend(ds):     
    return ds.dayofweek in (5, 6)

In [ ]:
# agregamos al set de entrenamiento
train['is_spring'] = train['ds'].apply(is_spring)
train['is_summer'] = train['ds'].apply(is_summer)
train['is_autumn'] = train['ds'].apply(is_autumn)
train['is_winter'] = train['ds'].apply(is_winter)
train['is_weekend'] = train['ds'].apply(is_weekend)
train['is_weekday'] = ~train['ds'].apply(is_weekend) 

In [ ]:
# agregamos al set de testeo
test['is_spring'] = test['ds'].apply(is_spring)
test['is_summer'] = test['ds'].apply(is_summer)
test['is_autumn'] = test['ds'].apply(is_autumn)
test['is_winter'] = test['ds'].apply(is_winter)
test['is_weekend'] = test['ds'].apply(is_weekend)
test['is_weekday'] = ~test['ds'].apply(is_weekend)

# test_prophet.shape
# test_prophet[test_prophet["is_weekend"]][0:100]

## Definimos función MAPE: error de porcentaje absoluto medio

In [ ]:
def mape(y_true, y_pred):
    """Error de porcentaje absoluto medio"""
    
    # conversión a vectores numpy
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    
    # Porcentaje de error
    pe = (y_true - y_pred) / y_true
    
    # valor absolutos
    ape = np.abs(pe)
    
    # Cuantificación del rendimiento en un solo nº
    mape = np.mean(ape)
    
    return f'{mape*100:.2f}%'

## Ajustando Hiper-Parámetros del modelo

__Time-Serie & Hyperparameter: ['Tuning'](https://www.kaggle.com/manovirat/timeseries-using-prophet-hyperparameter-tuning/notebook#HyperParameter-Tuning-using-ParameterGrid)__

<i>El siguiente bloque de código es para guardar 150 combinaciones posibles:</i><br>

```
    from sklearn.model_selection import ParameterGrid
    params_grid = {'seasonality_mode':('multiplicative','additive'),
                   'changepoint_prior_scale':[0.1,0.2,0.3,0.4,0.5],
                   'holidays_prior_scale':[0.1,0.2,0.3,0.4,0.5],
                   'n_changepoints' : [100,150,200]}

    grid = ParameterGrid(params_grid)
    print(type(grid))

    cnt = 0
    for p in grid:
        cnt = cnt+1

    print('Total de posibles modelos',cnt)
```

In [ ]:
## AJUSTA LOS VALORES ANTES DE CORRER 
'''
modelo_tuneado = Prophet(growth= 'linear',   
                         n_changepoints = 200, 
                         changepoint_range=0.95, 
                         yearly_seasonality = False,
                         weekly_seasonality=False,
                         daily_seasonality = False,                                                 
                         seasonality_mode = 'additive',
                         seasonality_prior_scale=10.0,     #rango recomendado: 0.01 to 10 
                         changepoint_prior_scale = 0.005,  #rango recomendado: 0.001 to 0.5                      
                         interval_width=0.8)
                         
Yearly F: 100 / Weakly F: 15 / Daily F: 7
13.52%
'''

modelo_tuneado = Prophet(n_changepoints = 200, 
                         changepoint_range=0.98, 
                         yearly_seasonality = False,
                         weekly_seasonality=False,
                         daily_seasonality = False,                                                 
                         seasonality_mode = 'additive',
                         seasonality_prior_scale=0.01,
                         changepoint_prior_scale = 0.3)

modelo_tuneado.add_seasonality(name='yearly', period=365.25, fourier_order = 15)

modelo_tuneado.add_seasonality(name='weekly_spring', 
                        period=7,
                        fourier_order = 5,
                        condition_name='is_spring')

modelo_tuneado.add_seasonality(name='weekly_summer', 
                        period=7,
                        fourier_order=5,
                        condition_name='is_summer')

modelo_tuneado.add_seasonality(name='weekly_autumn', 
                        period=7,
                        fourier_order=5,
                        condition_name='is_autumn')

modelo_tuneado.add_seasonality(name='weekly_winter', 
                        period=7,
                        fourier_order=5,
                        condition_name='is_winter')

modelo_tuneado.add_seasonality(name='daily_spring',  
                        period=1,
                        fourier_order=3, 
                        condition_name='is_spring')

modelo_tuneado.add_seasonality(name='daily_summer',  
                        period=1,
                        fourier_order=3,
                        condition_name='is_summer')

modelo_tuneado.add_seasonality(name='daily_autumn',  
                        period=1,
                        fourier_order=3,
                        condition_name='is_autumn')

modelo_tuneado.add_seasonality(name='daily_winter',  
                        period=1,
                        fourier_order=3,
                        condition_name='is_winter')

modelo_tuneado.add_seasonality(name='daily_weekend',  
                        period=1,
                        fourier_order=3,
                        condition_name='is_weekend')

modelo_tuneado.add_seasonality(name='daily_weekday',  
                        period=1,
                        fourier_order=3,
                        condition_name='is_weekday')
                        
# Feriados/días festivos
# modelo.add_country_holidays(country_name = 'AR')

## Nueva predicción y gráficos

In [ ]:
# fitting el modelo
modelo_tuneado.fit(train)

In [ ]:
#parte del dataframe en el que queremos hacer la prediccion
future = test.drop(['y'], axis=1)
# Prediciendo valores
tunning_forecast = modelo_tuneado.predict(future)

In [ ]:
# Gráfica para visualizar los puntos de cambios y la tendencia de la predicción
fig = modelo_tuneado.plot(tunning_forecast)
a = add_changepoints_to_plot(fig.gca(), modelo_tuneado, tunning_forecast)

In [ ]:
#graficando los componentes de Prophet.predict()
pd.plotting.register_matplotlib_converters()
_ = modelo_tuneado.plot_components(tunning_forecast)

In [ ]:
final_df = pd.concat((tunning_forecast['yhat'], test), axis = 1)
final_df = final_df[['ds', 'y', 'yhat']]
final_df

## Cross-Validation

In [ ]:
df_cv_t = cross_validation (modelo_tuneado, horizon = '180 days', period='90 days', initial='365 days')

In [ ]:
df_pm_t = performance_metrics(df_cv_t)

In [ ]:
fig_cv_t = plot_cross_validation_metric(df_cv_t, metric='mape')

## Graficamos los nuevos resultados obtenidos: Curva de test y de valores predecidos

In [ ]:
# create figure
fig = go.Figure()
fig.add_trace(go.Scatter(x=final_df.ds, y=final_df.y,
                         mode='lines',
                         name='Test - Ground Truth'))
fig.add_trace(go.Scatter(x=final_df.ds, y=final_df.yhat,
                         mode='lines', 
                         name='Test - Prediction'))

# adjust layout
fig.update_traces(line=dict(width=0.5))
fig.update_layout(title='Prophet Forecast of Hourly Energy Demand',
                  xaxis_title='Date & Time (yyyy/mm/dd hh:MM)',
                  yaxis_title='Energy Demand [MW]')
fig.show()

# quantify accuracy
print(f'MAPE for Prophet\'s predictions: {mape(final_df.y, final_df.yhat)}')

## Visualización de la primer y última semana (test vs previsión)

In [ ]:
# Longitud de intervalo
interval = 672

# Necesitamos adaptar al intervalo las variables a usarse, 
# dado que la predicción se hizo por intervalos de 24*365
x_true, y_true = test_prophet.iloc[:interval].ds, test_prophet.iloc[:interval].y
x_pred, y_pred = tunning_forecast.iloc[:interval].ds, tunning_forecast.iloc[:interval].yhat

# Grafica
fig = go.Figure()
fig.add_trace(go.Scatter(x=x_true, y=y_true,
                         mode = 'lines',
                         name = 'Test - Ground Truth'))
fig.add_trace(go.Scatter(x=x_pred, y=y_pred,
                         mode = 'lines',
                         name = 'Test - Prediction'))
# Ajustes varios sobre la grafica
fig.update_traces(line=dict(width=0.9))
fig.update_layout(title='Prophet: Pronóstico de las primeras 168 horas de Demanda',
                  xaxis_title='Date & Time (yyyy/mm/dd hh:MM)',
                  yaxis_title='Energy Demand [MW]')
fig.show()

# Eficacia 
print(f'MAPE para el intervalo en las primeras 168 horas: {mape(y_true, y_pred)}')

In [ ]:
# Longitud de intervalo
interval = -672

# Necesitamos adaptar al intervalo las variables a usarse, 
# dado que la predicción se hizo por intervalos de 24*365
x_true, y_true = test_prophet.iloc[interval:].ds, test_prophet.iloc[interval:].y
x_pred, y_pred = tunning_forecast.iloc[interval:].ds, tunning_forecast.iloc[interval:].yhat

# Grafica
fig = go.Figure()
fig.add_trace(go.Scatter(x=x_true, y=y_true,
                         mode = 'lines',
                         name = 'Test - Ground Truth'))
fig.add_trace(go.Scatter(x=x_pred, y=y_pred,
                         mode = 'lines',
                         name = 'Test - Prediction'))
# Ajustes varios sobre la grafica
fig.update_traces(line=dict(width=0.9))
fig.update_layout(title='Prophet: Pronóstico de las últimas 168 horas de Demanda',
                  xaxis_title='Date & Time (yyyy/mm/dd hh:MM)',
                  yaxis_title='Energy Demand [MW]')
fig.show()

# Eficacia 
print(f'MAPE para el intervalo en las últimas 168 horas: {mape(y_true, y_pred)}')

## Guardo los datos obtenidos en un archivo .csv

In [ ]:
# df_y = pd.DataFrame(forecast)
# df_y.rename(columns={'ds':'DateTime', 'y':'Demanda en [MW]', 'yhat': 'Demanda pronosticada en [MW]'}, inplace = True)
# df_y.set_index('DateTime')

In [ ]:
# df_y.to_csv('forecasting_prophet.csv', columns=['Demanda pronosticada en [MW]'], encoding='utf-8')

## Pasos a seguir

In [ ]:
# Probar con extensión de tiempo

In [ ]:
# Mejorar el seguimiento de la curva de pronostico con la de test para reducir el MAPE. 

In [ ]:
# Aplicar el modelo a los 4 medidores restantes